<a href="https://colab.research.google.com/github/minkyoJang/AI-BigDataStudy/blob/master/jmkDay23_Ques)Training_Neural_Network_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Neural Network 실습 
### Copyright (C) 2018  Cheonbok Park <cb_park@korea.ac.kr>


In [0]:
import torch 
import torch.nn as nn # 뉴럴네트워크는 이 nn 안에 있어요. 리니얼, rnn등 다양한거 안에 있어요. 이미 생성된거 사용하여 편리. 
import torch.nn.functional as F #tanh, relu, 일반적 파이토치 위한 뉴럴 function. 활성화 함수들이 여기에 있어요. 
import torchvision # 이미지 관련 처리, Pretrained Model 관련된 Package 입니다. 
import torchvision.datasets as vision_dsets #이미 주어진 데이터 셋 사용. 가지고 있는 일반적인 비전 데이터 셋을 넣어둔 것. 
import torchvision.transforms as T # 이미지 처리 (Vison) 관련된 transformation이 정의 되어 있습니다. 기본적 이미지는 RGB (0~256)인데 이건 값이 크니까 노말라이제이션해줘. 이거 해주는게 트랜스폼. 
import torch.optim as optim # pytorch 에서 정의한 수 많은 optimization function 들이 들어 있습니다. (옵티마이저. 아담!)
from torch.utils import data # 

# Process
- 런타임 유형: 하드웨어 유형 gpu로 변경하여 빨리 처리하게 합니다.

# MNIST Feed-forward Neural Network 

## Data Loader 불러오기
- 0과 1로 노말레이제이션 하셨어요
- 최대값1 최소0 민맥스하여 트랜스폼투 텐서
- 셔플링 필수** 
    - 100개 중에 10개 뽑을때 shuffle하면 순서가 바뀜. 
    - 트레이닝할땐 다양하게 돌려서 object function나와야 해서 shuffe=True
- 데이터 로더와 셋을 받습니다.

In [0]:

def MNIST_DATA(root='./',train =True,transforms=None ,download =True,batch_size = 32,num_worker = 1):

	print ("[+] Get the MNIST DATA")
	"""
  	torchvision.dataset 에는 우리가 많이 사용하는 데이터들을 쉽게 사용할 수 있도록 되어 있습니다. 
  	Machine Learning 에서 Hello world 라고 불리는 Mnist 데이터를 사용해 보겠습니다. 
  
  
	"""
	mnist_train = vision_dsets.MNIST(root = root,  #root 는 데이터의 저장 위치 입니다. 
									train = True, #Train 은 이 데이터가 train 데이터인지 아닌지에 대한 정보입니다. 
									transform = T.ToTensor(), # 얻어낸 데이터를 pytorch가 계산 할 수 있는 Tensor 로 변환해 줍니다. 
									download = True)  # 데이터를 다운로드 할지 여부를 물어봅니다. 
	mnist_test = vision_dsets.MNIST(root = root,
									train = False,  # Test Data를 가져오기에 Train =False 를 줘야 합니다. 
									transform = T.ToTensor(),
									download = True)
	"""
  	Data Loader 는 데이터와 batch size의 정보를 바탕으로 매 iteration 마다 주어진 데이터를 원하는 batch size 만큼 반환해주는 iterator입니다. 
  	* Practical Guide : Batch size 는 어느정도가 좋나요? -- 클 수록 좋다는 소리가 있습니다. 하지만 gpu memeory 사이즈 한계에 의해 기본적으로 batch size 가 
  	커질 수록 학습에 사용되는 gpu memory 사이즈가 큽니다. (Activation map을 저장해야 하기 때문입니다.) 기본적으로 2의 배수로 저장하는 것이 좋습니다.(Bit size 관련) 
  
	"""
	trainDataLoader = data.DataLoader(dataset = mnist_train,  # DataSet은 어떤 Data를 제공해 줄지에 대한 정보입니다. 여기서는 Training DATA를 제공합니다. 
									batch_size = batch_size, # batch size 정보를 꼭 줘야 합니다. 한 Batch 당 몇 개의 Data 를 제공할지에 대한 정보입니다. 
									shuffle =True, # Training의 경우 Shuffling 을 해주는 것이 성능에 지대한 영향을 끼칩니다. 꼭 True 를 줘야 합니다. 
									num_workers = 1) # num worker의 경우 데이터를 로드하는데 worker를 얼마나 추가하겠는가에 대한 정보입니다. 

	testDataLoader = data.DataLoader(dataset = mnist_test, # Test Data Loader 이므로 Test Data를 인자로 전달해줍니다.
									batch_size = batch_size, # 마찬가지로 Batch size 를 넣어줍니다. 
									shuffle = False, # shuffling 이 굳이 필요하지 않으므로 false를 줍니다. 
									num_workers = 1) #
	print ("[+] Finished loading data & Preprocessing")
	return mnist_train,mnist_test,trainDataLoader,testDataLoader

In [7]:
trainDset,testDset,trainDataLoader,testDataLoader= MNIST_DATA(batch_size = 32)  # Data Loader 를 불러 옵니다. 

0it [00:00, ?it/s]

[+] Get the MNIST DATA


9920512it [00:01, 8928217.07it/s]                            


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

...

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2566399.06it/s]                           
0it [00:00, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 53452.98it/s]            

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!
[+] Finished loading data & Preprocessing


## Train Function 
- (N,RGB, width, weight)
- (N, 1, 28,28)

- 에폭
    - 모든 데이터 돌때 하나의 에폭을 돌았다고 해요
    - 미니배치로 업데이트 하는 것== 이터레이션
    - 총 100개의 데이터 중에서 10번의 미니배치 사이즈를 하고 한번 에폭돌때는 몇 번의 이터레이션 돈거야? 10개
- 뎁스. 
    - RGB는 3, 일차원 색상은 1
    - cpu>gpu에 올려줘야해. 현재 우리 gpu로 작동하게 했으니
        - 그래서 cuda로 올려줘
- 옵티마이저: 그래디언트 값을 바탕으로 모델 웨이트 올려
    - 모델의 파라미터 가지고 있어서 처음엔 zero_grad해야해
        - 백프라그래하면 모델안에 저장이 되니까 축적되어 메모리 커짐.
        - 버퍼 플러쉬
- 뉴럴에 인풋넣고 아웃풋 내는데 이 아웃풋이 프레딕트
- 이 아웃푹과 정답label 비교
- 크리터리언으로 로스 비교. 
    - 크로스 엔트로피 로스를 사용할거에요. 클래시피케이션이니까. 
- 백워드 연산 실행
    - 로스를 바탕으로 백프로시작을 해. 
    - 순차적으로 각 파라미터에 그라디언트 구할 수 있음. 
- 옵티마이저.스텝
    - 가각 그래디언트에 대한 웨이트 업데이트하는 과정

In [0]:
def train_network(net,optimizer,trainloader):
  for epoch in range(4):  # loop over the dataset multiple times

      running_loss = 0.0 # running loss를 저장하기 위한 변수입니다. 
      for i, data in enumerate(trainloader, 0): # 한 Epoch 만큼 돕니다. 매 iteration 마다 정해진 Batch size 만큼 데이터를 뱉습니다. 
          # get the inputs
          inputs, labels = data # DataLoader iterator의 반환 값은 input_data 와 labels의 튜플 형식입니다. 
          inputs = inputs.cuda() # gpu에 데이터를 올립니다.
          labels = labels.cuda()
          # zero the parameter gradients
          optimizer.zero_grad()    #  현재 기존의 backprop을 계산하기 위해서 저장했던 activation buffer 를 비웁니다. Q) 이걸 안 한다면?

          # forward + backward + optimize
          outputs = net(inputs) # input 을 넣은 위 network 로 부터 output 을 얻어냅니다. 
          loss = criterion(outputs, labels) # loss fucntion에 주어진 target과 output 의 score를 계산하여 반환합니다. 
          loss.backward() # * Scalar Loss value를 Backward() 해주게 되면 주어진 loss값을 바탕으로 backpropagation이 진행됩니다. 
          optimizer.step() # 계산된 Backprop 을 바탕으로 optimizer가 gradient descenting 을 수행합니다. 

          # print statistics
          running_loss += loss.item()
          if i % 500 == 499:    # print every 2000 mini-batches
              print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 500))
              running_loss = 0.0

  print('Finished Training')

## Test Function
- 주의) model.eval
    - 실수하지 말아야 할 것! model.eval
        - 트레이닝 플레그를 false로 해야해요
        - 트레이닝 중인지 아니면 이벨인지에 대한 플레그를 갖고 있음. 
        - 모델(점)이발로 돌아오면 우리가 사용중인게 배치노말라이제이션 및 배치라고 하자. 
            - 트레이닝과 테스트일때 저 둘은 다르죠
            - 플래그를 넣어줘야해요. 그래서 이발은 테스트 모델일때 드랍 아웃 꺼주고 했던거죠***

In [0]:
def test(model,test_loader):
  model.eval() # Eval Mode 왜 해야 할까요?  --> nn.Dropout BatchNorm 등의 Regularization 들이 test 모드로 들어가게 되기 때문입니다. 
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data, target = data.cuda(), target.cuda()  # 기존의 train function의 data 처리부분과 같습니다. 
    output = model(data) 
    pred = output.max(1, keepdim=True)[1] # get the index of the max 
    correct += pred.eq(target.view_as(pred)).sum().item() # 정답 데이터의 갯수를 반환합니다. 

  test_loss /= len(test_loader.dataset)
  print('\nTest set:  Accuracy: {}/{} ({:.0f}%)\n'.format(
      correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))

## Neural Network  + Activation Function

## 간단한 Neural Network 를 만들어 봅시다. (1)
특징 : 2개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 30 + Activation Fucntion - Sigmoid 

Layer 2 - input: 30 output:10
          - 모델의 클래스갯수가 0에서 9니까 총 10개 그래서 10개 되야함

Cross Entropy Loss  + SGD optimizer 

In [0]:

class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() # nn.Module 생성자 호출 Q) 왜 필요할까요?
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30) # 리니어에서 인풋넣고, 아웃풋나오게 (인풋, 아웃풋)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        # view는 reshape 함수와 동일. 리니어를 하려면 하나의 디맨젼으로 쭉 펴줘야 하니까. -1은 자동으로 숫자 맞춰주라는 의미.
        # reshape=28*28> 30으로 리니어를 하고, 넌리니얼해줘야해(시그모이드)f.시그모이드로 액티베이션하여 시그모이드 거치게. 
        x = x.view(-1,28*28) # x.view함수는 주어진 인자의 크기로 해당 데이터의 크기를 반환합니다. 즉, (Batch_size,28,28) --> (Batch_size,28*28)로 변환합니다.
        x = F.sigmoid(self.fc0(x)) # 28*28 -> 30 -> Activation function 을 수행합니다. 즉, 레잉어 1번에 넣어서 아웃풋 30개 나오게 해 
        x = self.fc1(x)  # 30 -> 10 으로 10개의 Class에 대한 logit 값을 호출합니다. 
        return x
            # 로짓: 하나의 고양이(-2.7),개(2)등을 classification하게 된다면 softmax해줘야하죠. 그럼 고양이 0.01 개 0.99로 바꾸기 직전이 소프트 맥스.

#### Optimizer 
Optimizer 의 경우 기본적으로 torch.optim 안에 존재합니다. 다양한 optimziers 가 정의되어 있습니다. 

기본적으로 다음과 같은 구성을 따릅니다. optim.{Optimzier 이름}({Network Parameters},lr ={learning rate })

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 포인터로 가리켜서 그 그레디언트 기반으로 업데이트하려고 파라미터 줌. 
criterion = nn.CrossEntropyLoss() #소프트맥스한걸 아래에 주면 안됩니다.** 로짓을 넘겨야 해요******@@
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다.  Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 이 안에 소프트 맥스가 있음. 

In [12]:
train_network(mnist_net,optimizer,trainDataLoader) # 4 Epoch 정도 학습을 진행해봅니다. 

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,   500] loss: 2.309
[1,  1000] loss: 2.292
[1,  1500] loss: 2.276
[2,   500] loss: 2.252
[2,  1000] loss: 2.240
[2,  1500] loss: 2.226
[3,   500] loss: 2.204
[3,  1000] loss: 2.190
[3,  1500] loss: 2.177
[4,   500] loss: 2.149
[4,  1000] loss: 2.131
[4,  1500] loss: 2.115
Finished Training


In [0]:
test(mnist_net,testDataLoader) # Test 정확도를 출력해 봅니다. 

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



Test set:  Accuracy: 5958/10000 (60%)



### 간단한 Neural Network 를 만들어 봅시다. (2)
특징 : 2개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 30 + Activation Fucntion - tanh 

Layer 2 - input: 30 output:10

Cross Entropy Loss  + SGD optimizer 

- INIT은 레고 만들고
_ FORWARD로 만든다 생각**

In [0]:

class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() # nn.Module 생성자 호출 Q) 왜 필요할까요?
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(-1,28*28) # x.view함수는 주어진 인자의 크기로 해당 데이터의 크기를 반환합니다. 즉, (Batch_size,28,28) --> (Batch_size,28*28)로 변환합니다.
        x = F.tanh(self.fc0(x)) # 28*28 -> 30 -> Activation function 을 수행합니다.
        x = self.fc1(x)  # 30 -> 10 으로 10개의 Class에 대한 logit 값을 호출합니다. 
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [16]:
train_network(mnist_net,optimizer,trainDataLoader)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[1,   500] loss: 2.222
[1,  1000] loss: 2.067
[1,  1500] loss: 1.924
[2,   500] loss: 1.674
[2,  1000] loss: 1.552
[2,  1500] loss: 1.437
[3,   500] loss: 1.256
[3,  1000] loss: 1.176
[3,  1500] loss: 1.107
[4,   500] loss: 0.993
[4,  1000] loss: 0.941
[4,  1500] loss: 0.907
Finished Training


In [0]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 8327/10000 (83%)



### 간단한 Neural Network 를 만들어 봅시다. (3)
특징 : 2개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 30 + Activation Fucntion - Relu

Layer 2 - input: 30 output:10

Cross Entropy Loss  + SGD optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        # If the size is a square you can only specify a single number
        x = x.view(-1,28) #reshape
        x = F.relu(self.fc0(x))
        x = self.fc1(x)
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [22]:
train_network(mnist_net,optimizer,trainDataLoader)

RuntimeError: ignored

# sigmoid
- 지그재그 (0~1)
# tanh
- -1~1

#relu 
- 0또는 1이라 빠르다. 



In [0]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 8473/10000 (85%)



### Q) 성능차이가 존재하나요? 존재한다면 무슨 이유일까요?

### 간단한 Neural Network 를 만들어 봅시다. (4) 
특징 : 3개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 40 + Activation Fucntion - sigmoid 

Layer 2 - input: 40 output: 30

Layer 3 - input: 30 output : 10

Cross Entropy Loss  + SGD optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.fc0 = nn.Linear(28*28,40) # Layer 1
        self.fc1 = nn.Linear(40, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
      
        x = x.view(-1,28*28)
        x = F.sigmoid(self.fc0(x))# 시그모이드 레이어 3개 쌓았더니(아깐2개) 11프로 나왔네. 
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [25]:
train_network(mnist_net,optimizer,trainDataLoader) #

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,   500] loss: 2.317
[1,  1000] loss: 2.308
[1,  1500] loss: 2.302
[2,   500] loss: 2.299
[2,  1000] loss: 2.300
[2,  1500] loss: 2.299
[3,   500] loss: 2.299
[3,  1000] loss: 2.298
[3,  1500] loss: 2.297
[4,   500] loss: 2.297
[4,  1000] loss: 2.297
[4,  1500] loss: 2.297
Finished Training


In [0]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 1135/10000 (11%)



### Q) 학습이 잘 되나요???? 안 된다면 왜 안될까요?
- 시그모이드 레이어 3개 쌓았더니(아깐2개) 11프로 나왔네. 

### 간단한 Neural Network 를 만들어 봅시다. (5) 
특징 : 3개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 40 + Activation Fucntion - Relu 

Layer 2 - input: 40 output: 30

Layer 3 - input: 30 output : 10

Cross Entropy Loss  + SGD optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,40) #Layer 1 
        self.fc1 = nn.Linear(40, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
       
        x = x.view(-1,28*28)
        x = F.relu(self.fc0(x)) # Layer 1
        x = F.relu(self.fc1(x)) # Layer 2
        x = self.fc2(x)
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [28]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 2.305
[1,  1000] loss: 2.295
[1,  1500] loss: 2.284
[2,   500] loss: 2.258
[2,  1000] loss: 2.240
[2,  1500] loss: 2.211
[3,   500] loss: 2.146
[3,  1000] loss: 2.096
[3,  1500] loss: 2.040
[4,   500] loss: 1.918
[4,  1000] loss: 1.813
[4,  1500] loss: 1.713
Finished Training


In [29]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 6494/10000 (65%)



#### (4)와 차이가 존재하나요? 그렇다면 왜 그럴까요? (3) 이랑은 비교해보면 어떻나요? 
- 현재 렐루로 하니까 아까보다 loss 느리긴 하는데 많이 빠르진 않아요.
- 오티마이저보다 좋은거있쬬 디폴트로 생각하라고. 
    - 아담. 아담엔 모멘텀+아레베스플라빗(캐쉬)있다 생각하면ㄷ ㅙ

### 간단한 Neural Network 를 만들어 봅시다. (6) 
특징 : 3개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 40 + Activation Fucntion - Relu 

Layer 2 - input: 40 output: 30

Layer 3 - input: 30 output : 10

Cross Entropy Loss  + **Adam** optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,40) #Layer 1 
        self.fc1 = nn.Linear(40, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
       
        x = x.view(-1,28*28)
        x = F.relu(self.fc0(x)) # Layer 1
        x = F.relu(self.fc1(x)) # Layer 2
        x = self.fc2(x) # Layer 3 
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [36]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.684
[1,  1000] loss: 0.299
[1,  1500] loss: 0.260
[2,   500] loss: 0.199
[2,  1000] loss: 0.188
[2,  1500] loss: 0.170
[3,   500] loss: 0.137
[3,  1000] loss: 0.140
[3,  1500] loss: 0.140
[4,   500] loss: 0.111
[4,  1000] loss: 0.108
[4,  1500] loss: 0.111
Finished Training


In [37]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9661/10000 (97%)



### 간단한 Neural Network 를 만들어 봅시다. (7) Layer 를 줄여볼까요? 
특징 : 2개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 30 + Activation Fucntion - Relu 

Layer 2 - input: 30 output : 10

Cross Entropy Loss  + **Adam** optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30) #Layer 1 
        self.fc1 =  nn.Linear(30, 10) # Layer 2

    def forward(self, x):
       
        x = x.view(-1,28*28)
        x = F.relu(self.fc0(x)) # Layer 1
        x = self.fc1(x) # Layer 2
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [40]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.685
[1,  1000] loss: 0.324
[1,  1500] loss: 0.284
[2,   500] loss: 0.240
[2,  1000] loss: 0.224
[2,  1500] loss: 0.222
[3,   500] loss: 0.188
[3,  1000] loss: 0.191
[3,  1500] loss: 0.177
[4,   500] loss: 0.156
[4,  1000] loss: 0.163
[4,  1500] loss: 0.148
Finished Training


In [41]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9521/10000 (95%)



### 간단한 Neural Network 를 만들어 봅시다. (7) Batch Norm 을 줘 볼까요?***********
- 2디와 1디 있어요.
- 평균을 빼고 배리언스로 나눠서 정규화 시키고 스케일링과 시프팅 있어요
- 배치넘은 인풋 그대로 알려주고 하면ㄷ ㅙ

특징 : 2개의 Layer를 가지는 Neural Network 
<구성>  
Layer 1 - input:28*28 , output : 30 + Activation Fucntion - Relu  + Batch Norm

Layer 2 - input: 30 output : 10

Cross Entropy Loss  + **Adam** optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30) #Layer 1
        self.bn0 = nn.BatchNorm1d(30) # BatchNorm . 리니어 거치고 배ㅣ 넣을거야
        self.fc1 =  nn.Linear(30, 10) # Layer 2
    def forward(self, x):
       
        x = x.view(-1,28*28)
        x = F.relu(self.bn0(self.fc0(x))) # Layer 1.액티베이션 이전에만 배치넘 넣는건 아니야. 상황보고 넣으면 됩니다. 
        x = self.fc1(x) # Layer 2
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [44]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.893
[1,  1000] loss: 0.370
[1,  1500] loss: 0.301
[2,   500] loss: 0.238
[2,  1000] loss: 0.231
[2,  1500] loss: 0.209
[3,   500] loss: 0.183
[3,  1000] loss: 0.176
[3,  1500] loss: 0.176
[4,   500] loss: 0.154
[4,  1000] loss: 0.151
[4,  1500] loss: 0.159
Finished Training


In [45]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9601/10000 (96%)



### 간단한 Neural Network 를 만들어 봅시다. (8) 더 깊은 레이어에 Batch Norm 을 줘 볼까요?
- 리니어로 98~99프로 되면 거의 되고
- 컨벌류션 잘 주면 99프로 찍어요
- 엠니스트는 파이썬 계의 헬로우 월드. 잘 학습 되고요. 

특징 : 2개의 Layer를 가지는 Neural Network

<구성>  

Layer 1 - input:28*28 , output : 40 + Activation Fucntion - Relu + BatchNorm

Layer 2 - input: 40 output: 30 + Activation Fucntion - Relu  + BatchNorm

Layer 3 - input: 30 output : 10

Cross Entropy Loss  + **Adam** optimizer 

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,40) #Layer 1 
        self.bn0 = nn.BatchNorm1d(40) #BatchNorm1 
        self.fc1 = nn.Linear(40, 30) # Layer 2
        self.bn1 = nn.BatchNorm1d(30) #BatchNorm1 
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
       
        x = x.view(-1,28*28)
        x = F.relu(self.bn0(self.fc0(x))) # Layer 1
        x = F.relu(self.bn1(self.fc1(x))) # Layer 2
        x = self.fc2(x)
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [48]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.818
[1,  1000] loss: 0.277
[1,  1500] loss: 0.222
[2,   500] loss: 0.160
[2,  1000] loss: 0.159
[2,  1500] loss: 0.151
[3,   500] loss: 0.117
[3,  1000] loss: 0.127
[3,  1500] loss: 0.126
[4,   500] loss: 0.110
[4,  1000] loss: 0.116
[4,  1500] loss: 0.107
Finished Training


In [0]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9707/10000 (97%)



#### Batch Normalization 을 적용한 (7)과 (6)을 비교해보고 (8) 과 (5)를 비교해보면 어떻나요? 학습이 어떻게 달라졌을까요? 

## 왜 레이어 깊어질수록 성능이 좋아지나
- 좀더 컴플렉스한거 마들어서

## 배치넘하면 성능이 좋아지는 이유?
- 학습이 안정화되어 동 이터레이션 대비 성능 향상되는 것

#### 배치넘은 트레이닝을 안정화시켜서 빠른 성능 도달하게 하고, 깊은 레이어일수록 학습을 더 깊게 시킬 수 있다. 따라서 깊은 레이어일수록 배치넘으로 안정화 시킬 수 있을 것이다.

### Let's Do it - 성능을 한번 끝까지 높여볼까요~? 마음대로 한번 최고 성능을 찍어봅시다

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        
        self.fc0 = nn.Linear(28*28,16*5*5)
        self.bn0 = nn.BatchNorm1d(??)
        self.fc1 = nn.Linear(16 * 5 * 5, 84)
        self.bn1 = nn.BatchNorm1d(??)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
      
        # If the size is a square you can only specify a single number
        x = x.view(-1,28*28)
        x = F.relu(self.bn0(self.fc0(x)))
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.fc3(x)
        return x

In [0]:
mnist_net = MNIST_Net().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001)

In [0]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.469
[1,  1000] loss: 0.182
[1,  1500] loss: 0.149
[2,   500] loss: 0.105
[2,  1000] loss: 0.100
[2,  1500] loss: 0.096
[3,   500] loss: 0.062
[3,  1000] loss: 0.070
[3,  1500] loss: 0.071
[4,   500] loss: 0.049
[4,  1000] loss: 0.056
[4,  1500] loss: 0.056
Finished Training


In [0]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9791/10000 (98%)



## Practical Guide Pytorch nn.Sequential 



```
x = F.relu(self.bn0(self.fc0(x)))
x = F.relu(self.bn1(self.fc1(x)))
```
너무 복잡하지 않나요?  그냥 x = self.fc(x) 쉽게 해버리면 안 될까요?
- 큰 레이어 하나마다 블록을 만들어두고, 리니어 군집을 만들고.
- 레이어는 시퀀스하게 가기에 어떠한 리스트에 순서를 정합니다. 

Solution : nn.Sequential + 자매품 nn.ModuList


In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        
        layer_list = [] # 이 리스트에 모든 Layer 를 순차적으로 append 해보겠습니다. 
        layer_list.append(nn.Linear(28*28,40)) #Layer 1 
        layer_list.append(nn.ReLU())# activation function
        layer_list.append(nn.BatchNorm1d(40))#BatchNorm1 
        layer_list.append(nn.Linear(40, 30)) # Layer 2
        layer_list.append(nn.ReLU())# activation function
        layer_list.append(nn.BatchNorm1d(30)) #BatchNorm1 
        layer_list.append(nn.Linear(30, 10)) # Layer 3
        self.net  = nn.Sequential(*layer_list) # nn.Sequential 에 layer list를 넘겨 줍니다.시퀀스 패키지에 전달하면 이 리스트 순서대로 구성된 뉴럴 네트워크  레이어를 만들 수 있어요
    def forward(self, x):
       
        x = x.view(-1,28*28) #리쉐입한거 넣으면 아웃풋이 나와요 
        x = self.net(x) # 넣은 순서대로 적용이 됩니다. 
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [0]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.705
[1,  1000] loss: 0.275
[1,  1500] loss: 0.217
[2,   500] loss: 0.164
[2,  1000] loss: 0.157
[2,  1500] loss: 0.167
[3,   500] loss: 0.131
[3,  1000] loss: 0.144
[3,  1500] loss: 0.137
[4,   500] loss: 0.113
[4,  1000] loss: 0.120
[4,  1500] loss: 0.120
Finished Training


In [0]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9680/10000 (97%)



#### 연습해 봅시다 ! 

특징 : 2개의 Layer를 가지는 Neural Network <구성>

Layer 1 - input:28*28 , output : 30 + Activation Fucntion - Relu + Batch Norm

Layer 2 - input: 30 output : 10

Cross Entropy Loss + Adam optimizer

In [0]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        
        layer_list = [] # 이 리스트에 모든 Layer 를 순차적으로 append 해보겠습니다. 
        layer_list.append(nn.??(??,??)) #Layer 1 
        layer_list.append(nn.??())# ReLU activation function
        layer_list.append(nn.??(??)) #BatchNorm1 
        layer_list.append(nn.??(??, ??)) # Layer 2
        self.net  = nn.Sequential(*layer_list) # nn.Sequential 에 layer list를 넘겨 줍니다.
    def forward(self, x):
       
        x = x.view(-1,??)
        x = self.net(x) # 넣은 순서대로 적용이 됩니다. 
        return x

In [0]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [0]:
train_network(mnist_net,optimizer,trainDataLoader)

[1,   500] loss: 0.760
[1,  1000] loss: 0.383
[1,  1500] loss: 0.352
[2,   500] loss: 0.330
[2,  1000] loss: 0.329
[2,  1500] loss: 0.324
[3,   500] loss: 0.305
[3,  1000] loss: 0.308
[3,  1500] loss: 0.323
[4,   500] loss: 0.301
[4,  1000] loss: 0.302
[4,  1500] loss: 0.313
Finished Training


In [0]:
test(mnist_net,testDataLoader)


Test set:  Accuracy: 9204/10000 (92%)

